In [35]:
### Please READ ###

# This notebook is meant to be used to test if the reading pickle function works as well as inserting raw data into the database. This is not meant for production. Please execute `python backfill_db.py` as that is the intended pipeline.

In [ ]:
import pandas as pd
import os
import psycopg2

In [ ]:
cleaned_trades_df = pd.read_pickle("cleaned_trades_df__01-18-25.pkl")

# Load PostgreSQL connection details from docker-compose.yml environment variables
postgres_user = os.getenv('POSTGRES_USER', 'myuser')
postgres_password = os.getenv('POSTGRES_PASSWORD', 'mypassword')
postgres_host = os.getenv('POSTGRES_HOST', 'localhost')
postgres_port = os.getenv('POSTGRES_PORT', '5432')
postgres_db = os.getenv('POSTGRES_DB', 'mydb')


In [ ]:
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}")

cleaned_trades_df.to_sql('capitol_trades', engine, if_exists='replace', index=False)